# Tool Agent

In [1]:
!pip install python-dotenv langchain-community wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("MODEL_PATH")

In [2]:
import logging

from app.llm.generator import Generator
from app.llm.llamacpp.service import LlamaCppService
from app.state.final_answer.factory import FinalAnswerFactory

logging.basicConfig(level=logging.INFO)

llm = LlamaCppService(model_path=model_path, n_gpu_layers=-1)

final_answer_state = FinalAnswerFactory.get_node(Generator(
    service=llm,
    temperature=0.4,
))

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_m

In [3]:
import json

from pykka import ActorRef
from app.tools.adapter import ToolAdapter
from app.agent import BaseAgent
from app.state.node import Node, GenerationHandlerResponse
from app.memory.memory import Memory
from app.persona.persona import Persona
from app.llm.generator import Generator
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from typing import Dict, Any, Optional, List

tools = [LangchainWikipediaQueryRun(top_k_results=2, doc_content_chars_max=2048)]


class DelegationAgent(BaseAgent):
    def __init__(self, workers: List[ActorRef], persona: Persona, memory: Memory, nodes: List[Node],
                 default_initial_state: str, step_limit: int):
        super().__init__(persona, memory, nodes, default_initial_state, step_limit)
        memory.data.set("workers", workers)


def delegate_prompt_handler(persona: Persona, memory: Memory, _: Optional[Dict[str, Any]]) -> str:
    proxy_workers = [worker.proxy() for worker in memory.data.get("workers")]
    workers = [f"{worker.name.get()}: {worker.description.get()}" for worker in proxy_workers]
    return f'''{persona.prompt()}

Given the problem, determine which of your workers can help solve the problem and let them do it.

If none of your workers can help solve the problem, respond with "NONE: " and give the reason after.

Current Problem:
"""
{memory.data.get_current_message().content}
"""

Workers:
"""
{workers}
"""

Respond with only the name of the worker that can help solve the problem.'''


def delegate_generation_handler(
        response: str,
        memory: Memory,
        _: Optional[List[ToolAdapter]]) -> GenerationHandlerResponse:
    if "NONE: " in response:
        print(f"response: {response}")
        parse_response = response.split("NONE: ")[1].strip()
        return GenerationHandlerResponse(
            output=parse_response,
            next_state=final_answer_state.state_name,
            exclude_from_scratch_pad=True
        )

    worker = next((worker for worker in memory.data.get("workers") if worker.proxy().name.get() == response), None)
    if worker is not None:
        future = worker.ask(Query(goal=memory.data.get_current_message().content, delete_scratch_pad_on_finish=True))
        result = future.get()

        if result is not None:
            return GenerationHandlerResponse(
                output=f"Delegated to: {response}\nOutput: {result.output}",
                next_state=final_answer_state.state_name,
                exclude_from_scratch_pad=False
            )
        else:
            return GenerationHandlerResponse(
                output="Failed to get a response from the worker.",
                next_state=final_answer_state.state_name,
                exclude_from_scratch_pad=True
            )
    return GenerationHandlerResponse(
        output="No worker found with that name.",
        next_state=final_answer_state.state_name,
        exclude_from_scratch_pad=True
    )


delegate_node = Node(
    state_name="delegate",
    prompt_handler=delegate_prompt_handler,
    generation_handler=delegate_generation_handler,
    generator=Generator(
        service=llm,
        temperature=0.1,
    ),
    tools=tools,
)

delegation_persona = Persona(
    description="You're an a helpful assistant. Help the user solve their problem.")

In [4]:
from app.tools.langchain.yahoo_finance_news import LangchainYahooFinanceNews
from app.state.defaults import default_tools_handler
from app.state.node import Node
from app.memory.memory import Memory
from app.persona.persona import Persona
from app.llm.generator import Generator
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from typing import Dict, Any, Optional

tools = [
    LangchainWikipediaQueryRun(top_k_results=2, doc_content_chars_max=2048),
    LangchainYahooFinanceNews(top_k=2)
]


class SearchAgent(BaseAgent):
    name = "SearchAgent"
    description = "Specializes in searching for information."
    pass


def search_prompt_handler(persona: Persona, memory: Memory, tool_schemas: Optional[Dict[str, Any]]) -> str:
    messages_formatted = [f"{message.name}: {message.content}\n" for message in memory.data.get_all_messages()]
    return f'''{persona.prompt()}

Search for information using your tools to help solve the following problem for the user. Use any previous messages as context.

Current Problem:
"""
{memory.data.get_current_message().content}
"""

Previous Messages:
"""
{messages_formatted}
"""

Here are the schemas for the tools you have access to, pick only one:
"""
{tool_schemas}
"""

Respond with the JSON input for the tool of your choice to best solve the problem.'''


search_node = Node(
    state_name="search",
    prompt_handler=search_prompt_handler,
    generation_handler=default_tools_handler(
        next_state=final_answer_state.state_name,
        exclude_from_scratch_pad=False
    ),
    generator=Generator(
        service=llm,
        use_json_model=True,
        temperature=0.1,
    ),
    tools=tools,
)

search_persona = Persona(description="You're an expert researcher.")

In [5]:
search_agent = SearchAgent.start(persona=search_persona, memory=Memory(), nodes=[search_node, final_answer_state],
                                 default_initial_state="search", step_limit=5)
delegate_agent = DelegationAgent.start(persona=delegation_persona, memory=Memory(),
                                       nodes=[delegate_node, final_answer_state], workers=[search_agent],
                                       default_initial_state="delegate", step_limit=5)

In [8]:
from app.agent.messages import Query

future = delegate_agent.ask(Query(initial_state="delegate",
                                  goal="Find the latest news on NVDA stock."))
response = future.get(timeout=120)
print(json.dumps([step.model_dump() for step in response.steps], indent=2))
print(response.output)

Timeout: 120 seconds

In [8]:
future = delegate_agent.ask(Query(initial_state="delegate",
                                  goal="Can you research anything mentioned about it? Give me a summery."))
response = future.get(timeout=120)
print(json.dumps([step.model_dump() for step in response.steps], indent=2))
print(response.output)

[
  {
    "name": "delegate",
    "prompt": "You're an a helpful assistant. Help the user solve their problem.\n\nGiven the problem, determine which of your workers can help solve the problem and let them do it.\n\nIf none of your workers can help solve the problem, respond with \"NONE: \" and give the reason after.\n\nCurrent Problem:\n\"\"\"\nCan you research anything mentioned about it? Give me a summery.\n\"\"\"\n\nWorkers:\n\"\"\"\n['SearchAgent: Specializes in searching for information.']\n\"\"\"\n\nRespond with only the name of the worker that can help solve the problem.",
    "output": "Delegated to: SearchAgent\nOutput: The user requested information about \"NVDA stock\". After conducting research using the Wikipedia query tool, we found two relevant pages: ServiceNow and the Russell 3000 Index.\n\nServiceNow, Inc., the company behind NVDA (New Video Distribution Architecture) stock, is an American software company that develops a cloud computing platform to help manage digita

In [9]:
search_agent.stop()
delegate_agent.stop()

True

In [10]:
!pip install mermaid-py


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import mermaid as md
from mermaid.graph import Graph

graph: Graph = Graph('example', """
flowchart TD
    QueryNode{{"Query"}}
    subgraph DA["Delegation Agent"]
        DA_Start(Start)
        DA_DelegateNode{{"Delegate Node"}}
        DA_FinalState["Final State"]

        DA_Start --> DA_DelegateNode
        DA_DelegateNode --> DA_FinalState
    end

    subgraph SA["Search Agent"]
        SA_Start(Start)
        SA_SearchNode{{"Search Node"}}
        SA_FinalState["Final State"]

        SA_Start --> SA_SearchNode
        SA_SearchNode --> SA_FinalState
    end

    QueryNode --> DA_Start
    DA_DelegateNode -.->|Ask/Search| SA
    SA -.->|Response/Result| DA

    classDef agent fill:#f9f,stroke:#333,stroke-width:2px;
    class DA,SA agent;
""")
graphe: md.Mermaid = md.Mermaid(graph)
graphe